In [51]:
#Salmon Data NoteBook

! pip install pandas
! pip install openpyxl
!pip install rdflib
import pandas as pd
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit
from datetime import datetime

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [52]:
# Define the file paths
files = ['data/se_monthly_reports.csv']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

df = dfs[0]

In [53]:
df

,Monthly Report ID,Year,SEPA Site,Mortalities Kilograms,Actual Biomass on Site (tonnes),Submitted By,Licence Active at Report,Biomass Exceedence Tonnes,Feed Kilograms,Deltamethrin grams,...,Site Name,Active Licence Number,Operator,Maximum Biomass Allowed (tonnes),Receiving Water,National Grid Reference,Easting,Northing,Water Type,Local Authority
0,95415,01-Dec-2024,ACH1,0.00,0,Scottish Sea Farms Ltd,CAR/L/1015867,0.0,0,0.00,...,Achintraid (Kishorn Site 1),CAR/L/1015867,Scottish Sea Farms Ltd,1299,Loch Kishorn,NG 8247 3855,182466,838551,Seawater,Highland
1,95820,01-Dec-2024,FFMC41,1708.10,149,Scottish Sea Farms Ltd,CAR/L/1003486,0.0,0,0.00,...,"Achnacroish (Walters,Lismore East)",CAR/L/1003486,Scottish Sea Farms Ltd,999,Lynn of Lorn,NM 8479 4003,184790,740032,Seawater,Argyll and Bute
2,95421,01-Dec-2024,AIR1,21968.81,1840,Bakkafrost Scotland Ltd,CAR/L/1002917,0.0,446397,63.27,...,Aird Ardheslaig,CAR/L/1002917,Bakkafrost Scotland Ltd,2400,Loch Sheildaig,NG 7894 5617,178936,856170,Seawater,Highland
3,95427,01-Dec-2024,APT1,820.00,575,Dawnfresh Farming Ltd,CAR/L/1018068,0.0,243953,0.00,...,Aird Point (Etive 4),CAR/L/1018068,Dawnfresh Farming Ltd,1545,Loch Etive,NM 9916 3398,199157,733978,Seawater,Argyll and Bute
4,95670,01-Dec-2024,EAIT1,0.00,0,Cooke Aquaculture Scotland,CAR/L/1018464,0.0,0,0.00,...,Aith Voe East,CAR/L/1018464,Cooke Aquaculture Scotland,750,Aith Voe,HU 3490 5658,434904,1156578,Seawater,Shetland Islands
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82515,83973,01-Jan-2002,WAT1,691.00,249,Marine Harvest (Scotland) Ltd,WPC/N/62343,29.0,19352,0.00,...,Waternish,CAR/L/1002357,John MacPhee,500,Loch Dunvegan,NG 2540 5550,125400,855500,Seawater,Highland
82516,85662,01-Jan-2002,WFAR,323.00,230,Aquascot Seafarms Ltd,WPC/N/62019,0.0,4975,0.00,...,West Fara,CAR/L/1004229,Cooke Aquaculture Scotland,800,Scapa Flow,ND 3320 9523,331963,995227,Seawater,Orkney Islands
82517,86294,01-Jan-2002,WHI1,74.00,340,North Atlantic Salmon Ltd,WPC/N/70181,0.0,42921,0.00,...,Whitesness,WPC/N/70181,North Atlantic Salmon Ltd,400,Vaila Sound,HU 2430 4720,424300,1147200,Seawater,Shetland Islands
82518,81187,01-Jan-2002,VAT1,0.00,488,Aquascot Seafarms Ltd,WPC/N/61968(02),0.0,67975,0.00,...,Wick of Vatsetter,CAR/L/1004201,Cooke Aquaculture Scotland,1999,Colgrave Sound,HU 53489 90298,453489,1190298,Seawater,Shetland Islands


In [54]:
# Select specific columns
df = df[[
    'Monthly Report ID',
    'Year',
    'Mortalities Kilograms',
    'Actual Biomass on Site (tonnes)',
    'Biomass Exceedence Tonnes',
    'Feed Kilograms',
    'Maximum Biomass Allowed (tonnes)',
    'Water Type',
    'Local Authority'
]]

# Remove rows with any null values
df = df.dropna()

# Rename columns
df = df.rename(columns={
    'Monthly Report ID': 'SampleNumber',
    'Year': 'Date',
    'Mortalities Kilograms': 'MortalitiesKilograms',
    'Actual Biomass on Site (tonnes)': 'ActualBiomass',
    'Biomass Exceedence Tonnes': 'BiomassExceedenceTonnes',
    'Feed Kilograms': 'FeedKilograms',
    'Maximum Biomass Allowed (tonnes)': 'MaximumBiomassAllowed',
    'Water Type': 'WaterType',
    'Local Authority': 'LocalAuthority'
})
# Clean spaces inside 'LocalAuthority' values 
df['LocalAuthority'] = df['LocalAuthority'].str.replace(' ', '')

# Add static columns
df['Country'] = 'Scotland'
df["Lab_Reference"] = "1"


In [55]:
df

,SampleNumber,Date,MortalitiesKilograms,ActualBiomass,BiomassExceedenceTonnes,FeedKilograms,MaximumBiomassAllowed,WaterType,LocalAuthority,Country,Lab_Reference
0,95415,01-Dec-2024,0.00,0,0.0,0,1299,Seawater,Highland,Scotland,1
1,95820,01-Dec-2024,1708.10,149,0.0,0,999,Seawater,ArgyllandBute,Scotland,1
2,95421,01-Dec-2024,21968.81,1840,0.0,446397,2400,Seawater,Highland,Scotland,1
3,95427,01-Dec-2024,820.00,575,0.0,243953,1545,Seawater,ArgyllandBute,Scotland,1
4,95670,01-Dec-2024,0.00,0,0.0,0,750,Seawater,ShetlandIslands,Scotland,1
...,...,...,...,...,...,...,...,...,...,...,...
82515,83973,01-Jan-2002,691.00,249,29.0,19352,500,Seawater,Highland,Scotland,1
82516,85662,01-Jan-2002,323.00,230,0.0,4975,800,Seawater,OrkneyIslands,Scotland,1
82517,86294,01-Jan-2002,74.00,340,0.0,42921,400,Seawater,ShetlandIslands,Scotland,1
82518,81187,01-Jan-2002,0.00,488,0.0,67975,1999,Seawater,ShetlandIslands,Scotland,1


In [56]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')

df['FlooredDate'] = pd.to_datetime(df['Date'], errors='coerce', infer_datetime_format=True)

df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df = df.drop(['Date'], axis=1)


C:\Users\saban\AppData\Local\Temp\ipykernel_18608\364509590.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['FlooredDate'] = pd.to_datetime(df['Date'], errors='coerce', infer_datetime_format=True)


In [57]:
# Save file with the current date
current_date = datetime.today().strftime('%Y-%m-%d')
base_filename = "barometer_combined"
folder_path = "output/CleanedData/"
new_filename = f"{folder_path}{base_filename}_{current_date}.csv"

df.to_csv(new_filename, index=False)

In [58]:
import pandas as pd
from rdflib import Graph, URIRef, Literal, RDF, RDFS, Namespace
import urllib.parse

# Define your namespaces
g = Graph()
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

g = Graph()

LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
decide = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/decide#")

# Bind prefixes!
g.bind("LHO", LHO)
g.bind("decide", decide)
g.bind("rdfs", RDFS)
g.bind("rdf", RDF)

# Loop over only the first 10 rows
for index, row in df.head(10).iterrows():
    SampleNumber = row["SampleNumber"]
    FlooredDate = row["FlooredDate"]
    MortalitiesKilograms = row["MortalitiesKilograms"]
    ActualBiomass = row["ActualBiomass"]
    Country = row["Country"]
    LocalAuthority = row["LocalAuthority"]
    FeedKilograms = row["FeedKilograms"]
    WaterType = row["WaterType"]

    # Create a unique URI for each sample
    SalmonSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab1SalmonSample_{index}")
    
    # Add sample type assertion
    g.add((SalmonSample_uri, RDF.type, LHO.SalmonSample))
    description = "An individual representing a sample from a Salmon."
    g.add((SalmonSample_uri, RDFS.comment, Literal(description, lang="en")))
    
    # MortalitiesKilograms
    if pd.notnull(MortalitiesKilograms):
        MortalitiesKilograms_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{MortalitiesKilograms}")
        g.add((SalmonSample_uri, LHO.CS61, MortalitiesKilograms_uri))
        g.add((MortalitiesKilograms_uri, RDF.type, decide.Mortality))
        description = "An individual sample representing Salmon Mortality."
        g.add((MortalitiesKilograms_uri, RDFS.comment, Literal(description, lang="en")))
        
    # ActualBiomass
    if pd.notnull(ActualBiomass):
        ActualBiomass_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{ActualBiomass}")
        g.add((SalmonSample_uri, LHO.CS56, ActualBiomass_uri))
        g.add((ActualBiomass_uri, RDF.type, decide.ActualBiomass))
        description = "An individual representing Actual biomass of salmon."
        g.add((ActualBiomass_uri, RDFS.comment, Literal(description, lang="en")))
        
    # Country
    if pd.notnull(Country):
        Country_clean = urllib.parse.quote(Country.strip())
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country_clean}")
        g.add((SalmonSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Country class
        g.add((Country_uri, RDFS.label, Literal("Scotland")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    # LocalAuthority
    if pd.notnull(LocalAuthority):
        LocalAuthority_clean = urllib.parse.quote(LocalAuthority.strip())
        LocalAuthority_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{LocalAuthority_clean}")
        g.add((SalmonSample_uri, LHO.CS60, LocalAuthority_uri))
        g.add((LocalAuthority_uri, RDF.type, LHO.LocalAuthority))
        description = "An individual representing different Local Authorities in UK."
        g.add((LocalAuthority_uri, RDFS.comment, Literal(description, lang="en")))
    
    # WaterType
    if pd.notnull(WaterType):
        WaterType_clean = urllib.parse.quote(WaterType.strip())
        WaterType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{WaterType_clean}")
        g.add((SalmonSample_uri, LHO.CS57, WaterType_uri))
        g.add((WaterType_uri, RDF.type, LHO.WaterType))
        description = "An individual that represents the water type for salmon."
        g.add((WaterType_uri, RDFS.comment, Literal(description, lang="en")))


    if "FlooredDate" in row and not pd.isnull(row["FlooredDate"]):
        Date = str(row["FlooredDate"]).strip()
        Date = Date.split()[0]  # Get only date part
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
        g.add((SalmonSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
        description = "An individual representing the sampling date."
        g.add((Date_uri, RDFS.comment, Literal(description, lang="en")))


# Serialize RDF
rdf_output_file = "outputSalmon.rdf"
g.serialize(destination=rdf_output_file, format="xml")

# Also serialize as Turtle and print
turtle_data = g.serialize(format="turtle")
print(turtle_data)


@prefix LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#> .
@prefix decide: <http://www.purl.org/decide/LiveStockHealthOnto/decide#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

LHO:Lab1SalmonSample_0 a LHO:SalmonSample ;
    LHO:CS56 LHO:0 ;
    LHO:CS57 LHO:Seawater ;
    LHO:CS60 LHO:Highland ;
    LHO:CS61 LHO:0.0 ;
    LHO:hasCountry LHO:Scotland ;
    LHO:hasDate LHO:2024-12-01 ;
    rdfs:comment "An individual representing a sample from a Salmon."@en .

LHO:Lab1SalmonSample_1 a LHO:SalmonSample ;
    LHO:CS56 LHO:149 ;
    LHO:CS57 LHO:Seawater ;
    LHO:CS60 LHO:ArgyllandBute ;
    LHO:CS61 LHO:1708.1 ;
    LHO:hasCountry LHO:Scotland ;
    LHO:hasDate LHO:2024-12-01 ;
    rdfs:comment "An individual representing a sample from a Salmon."@en .

LHO:Lab1SalmonSample_2 a LHO:SalmonSample ;
    LHO:CS56 LHO:1840 ;
    LHO:CS57 LHO:Seawater ;
    LHO:CS60 LHO:Highland ;
    LHO:CS61 LHO:21968.81 ;
    LHO:hasCountry LHO:Scotland ;
    LHO:hasDate LHO:2024-12-0

In [59]:
#Define the original path
ontology_file = "Ontology/LiveStockHealthOntology.owl"
g.parse(ontology_file, format="xml")

g.serialize(destination='output/RDFoutputSalmon.owl', format="xml")


<Graph identifier=N479f11bd83204d5190c79227aa296b3a (<class 'rdflib.graph.Graph'>)>

In [60]:
query = """
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT
  (strafter(str(?Sample), "#") AS ?SampleName)
  (strafter(str(?MortalitiesKilograms), "#") AS ?MortalitiesKilogramsValue)
  (strafter(str(?ActualBiomass), "#") AS ?ActualBiomassValue)
  (strafter(str(?WaterType), "#") AS ?WaterTypeName)
  (strafter(str(?Country), "#") AS ?CountryName)
  (strafter(str(?LocalAuthority), "#") AS ?LocalAuthorityName)
  (strafter(str(?Date), "#") AS ?DateValue)
WHERE {
  ?Sample rdf:type LHO:SalmonSample .
  OPTIONAL { ?Sample LHO:CS61 ?MortalitiesKilograms . }
  OPTIONAL { ?Sample LHO:CS56 ?ActualBiomass . }
  OPTIONAL { ?Sample LHO:CS60 ?WaterType . }
  OPTIONAL { ?Sample LHO:hasCountry ?Country . }
  OPTIONAL { ?Sample LHO:CS57 ?LocalAuthority . }
  OPTIONAL { ?Sample LHO:hasDate ?Date . }
}
"""


# Execute the query
results = g.query(query)

# Convert the results to a Pandas dataframe
data = []
for row in results:
    data.append(list(row))

# Correct column names for Salmon!
df = pd.DataFrame(data, columns=[
    "SampleName",
    "MortalitiesKilogramsValue",
    "ActualBiomassValue",
    "WaterTypeName",
    "CountryName",
    "LocalAuthorityName",
    "DateValue"
])

# Display the dataframe
print(df)


           SampleName MortalitiesKilogramsValue ActualBiomassValue  \
0  Lab1SalmonSample_0                       0.0                  0   
1  Lab1SalmonSample_1                    1708.1                149   
2  Lab1SalmonSample_2                  21968.81               1840   
3  Lab1SalmonSample_3                     820.0                575   
4  Lab1SalmonSample_4                       0.0                  0   
5  Lab1SalmonSample_5                       0.0                  0   
6  Lab1SalmonSample_6                    5846.0               1815   
7  Lab1SalmonSample_7                   10611.0                798   
8  Lab1SalmonSample_8                       0.0                  0   
9  Lab1SalmonSample_9                       0.0                  0   

     WaterTypeName CountryName LocalAuthorityName   DateValue  
0         Highland    Scotland           Seawater  2024-12-01  
1    ArgyllandBute    Scotland           Seawater  2024-12-01  
2         Highland    Scotland       